# Investigating and Correcting Bias in NLP Models

Welcome to the Investigating and correcting bias in word vectors for NLP models, an assignment of CSCM21 (Abuses Bias and Blessings of Data) delivered by Dr Matt Roach. Here you will gain experience in using investigating and correcting bias in algorithms. Specifically in this lab we will look at a method of converting unstructured text into numerical representations (vectors) that also embed semantic understanding into the numerical feature space. This is commonly known as word2vec, and used in many computational Natural Language Processing (NLP) tasks, such as: text prediction, document topic classification, language translation and many more.

State of the art performance in NLP research has recently seen a transition of focus from rule-based or word-frequency probabilistic models to probabilistic models based on word-vector representations within the last decade. These types of models have made great strides in our ability to learn semantic relationships between words allowing for much richer modelling of natural language that has been possible with rule-based and word-frequency based approaches. One such example is the 'GloVe' word embeddings by Pennington <i> et al</i> [1], in this lab we will be using a word embedding model trained with the GloVe method.
    
Moreover, the world has seen a growth in large pre-trained models based on this new breed of NLP algorithms, and as such, there is a real danger that developer could adopt and embed these models in software products and services without being aware of the possible unintended consequences. You will be taking a look at how you might detect bias in word vector models and a method we could use to de-bias the model post-training presented by Bolukbasi <i>et al.</i> in titled <i>"Man is to Computer Programmer as Woman is to Homemaker? Debiasing Word Embeddings"</i> [2]


[1] J. Pennington, R. Socher, and C. D. Manning, “GloVe: Global vectors for word representation,” in EMNLP 2014  Conference on Empirical Methods in Natural Language Processing, Proceedings of the Conference, 2014 [Link to Paper](https://nlp.stanford.edu/projects/glove/).


[2] T. Bolukbasi, K.-W. Chang, J. Zou, V. Saligrama, and A. Kalai, “Man is to Computer Programmer as Woman is to Homemaker? Debiasing Word Embeddings,” Jul. 2016.[Link to Paper](https://papers.nips.cc/paper/6228-man-is-to-computer-programmer-as-woman-is-to-homemaker-debiasing-word-embeddings.pdf).

[3] This workbook is an based on one created by Andrew Ng and reuses some material with thanks.

## Learning Objectives:

- Investigate word similarity a pre-trained word embedding model.
- Use word embeddings to solve word analogy problems.
- Design counter factual word analogies to detect bias in the word embedding model.
- Modify the word embedding model to reduce its gender bias.
- Critically reflect on the effectiveness and limitations of the de-bais methods used.

Run the following cell to load the packages you will need.

In [ ]:
import sys
import os
import tensorflow as tf
from platform import python_version 

#------------- Load in some helpful packages we will use for the lab -------------------------

import numpy as np
from w2v_utils import * # included in the lab download file.
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.manifold import TSNE


#------------- Print versions an paths for potencial debugging -------------------------

print('Python version:', python_version()) # print the version of python the script is using
print('Python path:', sys.executable) # print the path for the python executable the script is using
print('Working Directory:', os.getcwd()) # print the current working directory 
print ("TensorFlow version: " + tf.__version__) # print the tensorflow version


# 1. Vectorised Word Embeddings Models

A reminder from work covered in the lecture, word vectors are generated using supervised training in the form of an auto-encoder.

<center><img src="images/w2v.png" width="800px"/></center>

<caption><center> <b>Figure</b> 1:  Word2Vec is a 3 layer architecture, with an input layer size equal to the size of the vocabulary, hidden layer of size of the embedding vector (typically 200-300 in this lab we will use a model based on 50) and output size again equal to the vocab size.</center></caption>
  
The input to the network is a 1-hot vector (a vector which all values are 0 except the value corresponding to the input word). This essentially acts as a mask on the embedding space, so that only the input word outputs nonzero activations from the embedding space through to the output layer. The output layer then predicts expected word based on the input words. The training objective function is a simple measure of the prediction performance. For the workbook we will work on the word embeddings - that are the outputs of the hidden layer from above, for each word - post training.


## 1.1 Load the Word Embedding Vectors. 

In this section, we will use 50-dimensional GloVe vectors to represent words.  We will not need to train the model or infact generate the vectors for each word this part has already been done. 

In the following line of code we will load a file "glove.6B.50d.txt" and stores the contents in python variables `word_to_vec_map` and `words`.

In [ ]:
words, word_to_vec_map = read_glove_vecs('data/glove.6B.50d.txt')

Above you have loaded:
- `words`: set of words in the vocabulary.
- `word_to_vec_map`: a python dictionary that contains the word GloVe vector representation for each word.
 - note the index to retrieve a single word vector from `word_to_vec_map` is the word itself i.e. to get the vector for mother you can use the syntax `word_to_vec_map[mother]`.

Lets take a look at the loaded sets of words and word vectors in the cell below investigate words that are in the models and take a look at their word vectors. 

**You need to:**
- find out the number of words in our vocabulary, i.e. print the number of elements (words) in `words`. Hint use `len`.
- print the vector for the word "farther".
- investigate the word vector for a small number (approx. 5) of different words. 
 - Note print each example to the note book (don't overwrite, in order to present examples of your exploration).

In [ ]:
### START CODE ###
num_words = None
print(None)
### END CODE ###

### START CODE ###
word = None
print(None)
### END CODE ###

## 1.2  Visualising  Word Vectors

In this next section we will investigate the semantic meaning embedded in the word vectors. To do this you will presenting example word vectors. The vector element values will be visuliased using a colour from the jet color map range illustrated below from blue (low) to red (high), the maximum and minimum range of the color maps is normalised to the max and min vector componet value contined in all the vectors in `word_to_vec_map`.

<center><img src="images/jet_colour.png" style="width:400px"></center>
    
<center><caption> <b>Figure 2:</b> Jet Colour Range (Left/Blue Least, Right/Red Most)</caption></center>

**Your tasks is to:**
    
- Calucate the maximum and minimum vector component values - buy running the cell below


In [ ]:
def viz_vectors(word_to_vec_map, vec_max, vec_min, word_1, word_2, word_3):
    """
    This funtion accepts three words and returns a figure handle for a plot.
    
    Inputs:
    word_to_vec_map - data dictionary conatin a map form work 'keys' to vectors
    vec_max  - a float containg maximum vector element value in dictionary
    vec_min - a float containg lowest vector element value in dictionary
    word_1 - string containing a word 
    word_2 - string containing a word
    word_3 - string containing a word
    
    Outputs:
    fig - a handle to a plot
    """

    wordvec_1 = word_to_vec_map[word_1]
    wordvec_2 = word_to_vec_map[word_2]
    wordvec_3 = word_to_vec_map[word_3]
    view_wordvec_1 = np.reshape(wordvec_1, (1, 50))
    view_wordvec_2 = np.reshape(wordvec_2, (1, 50))
    view_wordvec_3 = np.reshape(wordvec_3, (1, 50))

    fig = plt.figure(figsize = (15,2))

    ax1 = fig.add_subplot(311)
    ax1.axes.get_yaxis().set_visible(False)
    ax1.set_title(word_1)
    ax1.imshow(view_wordvec_1, interpolation='nearest', cmap=cm.jet, vmin=vec_min, vmax=vec_max)

    ax2 = fig.add_subplot(312)
    ax2.axes.get_yaxis().set_visible(False)
    ax2.imshow(view_wordvec_2, interpolation='nearest', cmap=cm.jet, vmin=vec_min, vmax=vec_max)
    ax2.set_title(word_2)
    
    ax3 = fig.add_subplot(313)
    ax3.axes.get_yaxis().set_visible(False)
    ax3.set_title(word_3)
    ax3.imshow(view_wordvec_3, interpolation='nearest', cmap=cm.jet, vmin=vec_min, vmax=vec_max)
    
    return(fig)

def vec_max_min(word_to_vec_map):
    vec_max = 0
    vec_min = 0

    for key in word_to_vec_map:
        if np.amax(word_to_vec_map[key]) > vec_max:
            vec_max = np.amax(word_to_vec_map[key])
        if np.amin(word_to_vec_map[key]) < vec_min:
            vec_min = np.amin(word_to_vec_map[key])

    return(vec_max,vec_min)

vec_max, vec_min = vec_max_min(word_to_vec_map)

print("the minimun vector componet value in the whole vocablray is: ", vec_min)
print("the maximum vector componet value in the whole vocablray is: ", vec_max)


**Expected Output**:

<table>
    <tr>
        <td>
            minimun vector componet value
        </td>
        <td>
         -5.4593
        </td>
    </tr>
        <tr>
        <td>
            maximum vector componet value
        </td>
        <td>
         5.3101
        </td>
    </tr>
</table>



**In the follwing cell your tasks is to:**
    
-  Visualise an addtinal three sets of three vectors where two vectors are similar semantically and have similar patterns in the vector visulisation and a third that is related but different in both semantic meaning and a difference can been seen in the vector visulisation, you can do this by calling the function `viz_vectors`.

In [ ]:

fig = viz_vectors(word_to_vec_map, vec_max, vec_min, 'man', 'woman', 'person')
plt.show()

##### START CODE #######

#Step 1: vizulise first set of three words
fig = None
plt.show()

#Step 1: vizulise first set of three words
fig = None
None

#Step 1: vizulise first set of three words
None

##### END CODE #########

## 1.3 Viewing Word Clusters in the Vector Space

In this section you will view the vectors plotted in the feature space, since our feature space is 50 dimensions we will use a dimension reduction technique to be able to plot the vectors in a 2-dimensional space. This technique compresses the 50 dimension into 2 whilst trying to retain the most significant information. Although is inevitable this technique looses discriminatory information we can still visualise semantic relationships in the vector space, as illustrated in the figure below.

<center><img src="images/Country_Capital_Vectors.png" width="500px"/></center>

<caption><center> <b>Figure 3:</b> Visulisation of vectors in 2-dimentional space, the word vector positions are marked by a red <span style="color:red"> X </span> the dotted lines are representing the semantic relationship between word vectors.</center></caption>

- run the cell below to load a function to reduce the feature space and plot visualise word vectors,


In [ ]:
def tsne_plot(model):
    """
    Creates and TSNE model and plots it
    """
    
    labels = []
    tokens = []

    for word in model:  #.wv.vocab
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=2, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])

    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()


**Your task it to:**

- run the code below to see the relationships between counties and their capitals
- create two new words lists that create three cluster of words in the feature space where the word within clusters have a semantic relationship and the words across clusters do not.
- there is an example already coded, make sure too add your code and plot (do not overwrite)
    

In [ ]:
word_list = ('china','bejing', 'japan', 'tokyo', 'italy', 'rome', 'spain', 'madrid', 'uk', 'london'
             ,'belgium', 'brussels', 'portugal', 'lisbon', 'russia','moscow', 'thailand','bangkok')

new_dict = {k:word_to_vec_map[k] for k in word_list if k in word_to_vec_map}

tsne_plot(new_dict)

word_list =('car' ,'bus','airplane', 'lorry','run','walk','jog','sprint', 'green', 'red' ,'blue', 'yellow')

new_dict = {k:word_to_vec_map[k] for k in word_list if k in word_to_vec_map}

tsne_plot(new_dict)

#### START CODE #####

None

### END CODE ######


# 2.  Measuring Word Similarity in Vector Space

Next we will generate a way of measuring the similarity of embedding vectors of two given words. We will use the cosine similarity measure since it takes into account the angle of the vector not just the magnitude as it is the angle of the vector that contains most of the semantic meaning. Given two vectors $u$ and $v$, cosine similarity is defined as follows: 

$$\text{CosineSimilarity(u, v)} = \frac {u . v} {||u||_2 ||v||_2} = cos(\theta) \tag{1}$$

where $u.v$ is the dot product (or inner product) of two vectors, $||u||_2$ is the norm (or length) of the vector $u$, and $\theta$ is the angle between $u$ and $v$. This similarity depends on the angle between $u$ and $v$. If $u$ and $v$ are very similar, their cosine similarity will be close to 1; if they are dissimilar, the cosine similarity will take a smaller value. 

<center><img src="images/cosine_sim.png" width="800px"/></center>

<caption><center> <b>Figure 4</b>: The cosine of the angle between two vectors is a measure of how similar they are</center></caption>

You need to:
- Implement the function `cosine_similarity()` to evaluate similarity between word vectors.

**Note:** The norm of $u$ is defined as $\vert\vert u\vert\vert_2 = \sqrt{\sum_{i=1}^{n} u_i^2}$

**Hint** There a many maths functions provided by the numpy library imported in to this notebook as `np` so you can use functions to calculate dot product, square root, L2-norm etc of vectors. e.g. `np.dot(u, v)` 

In [ ]:
def cosine_similarity(u, v):
    """
    Cosine similarity reflects the degree of similariy between u and v
        
    Arguments:
        u -- a word vector of shape (n,)          
        v -- a word vector of shape (n,)

    Returns:
        cosine_similarity -- the cosine similarity between u and v defined by the formula above.
    """
        
    ### START CODE HERE ###
    #Step 1: Compute the dot product between u and v
    dot = None
    #Step 2: Compute the L2 norm of u (≈1 line)
    norm_u = None
    
    #Step 3: Compute the L2 norm of v (≈1 line)
    norm_v = None
    
    # Compute the cosine similarity defined by formula (1) (≈1 line)
    cosine_similarity = None
    ### END CODE HERE ###
    
    return cosine_similarity

## 2.1 Testing your Similarity Function
Run the code below to test your Similarity function `cosine_similarity()`.

In [ ]:
father = word_to_vec_map["father"]
mother = word_to_vec_map["mother"]
ball = word_to_vec_map["ball"]
crocodile = word_to_vec_map["crocodile"]
france = word_to_vec_map["france"]
italy = word_to_vec_map["italy"]
paris = word_to_vec_map["paris"]
rome = word_to_vec_map["rome"]

print("cosine_similarity(father, mother) = ", cosine_similarity(father, mother))
print("cosine_similarity(ball, crocodile) = ",cosine_similarity(ball, crocodile))
print("cosine_similarity(france - paris, rome - italy) = ",cosine_similarity(france - paris, rome - italy))

**Expected Output**:

<table>
    <tr>
        <td>
            **cosine_similarity(father, mother)** =
        </td>
        <td>
         0.890903844289
        </td>
    </tr>
        <tr>
        <td>
            **cosine_similarity(ball, crocodile)** =
        </td>
        <td>
         0.274392462614
        </td>
    </tr>
        <tr>
        <td>
            **cosine_similarity(france - paris, rome - italy)** =
        </td>
        <td>
         -0.675147930817
        </td>
    </tr>
</table>

After you get the correct expected output, please feel free to modify the inputs and measure the cosine similarity between other pairs of words! Playing around the cosine similarity of other inputs will give you a better sense of how word vectors behave. 

# 2.2 - Word Analogy Task

In the word analogy task, we complete the sentence <font color='brown'>"*a* is to *b* as *c* is to **____**"</font>. An example is <font color='brown'> '*man* is to *woman* as *king* is to *queen*' </font>. In detail, we are trying to find a word *d*, such that the associated word vectors $e_a, e_b, e_c, e_d$ are related in the following manner: $e_b - e_a \approx e_d - e_c$. We will measure the similarity between $e_b - e_a$ and $e_d - e_c$ using cosine similarity. 

**Exercise**: Complete the code below to be able to perform word analogies!

In [ ]:
# complete_analogy

def complete_analogy(word_a, word_b, word_c, word_to_vec_map):
    """
    Performs the word analogy task as explained above: a is to b as c is to ____. 
    
    Arguments:
    word_a -- a word, string
    word_b -- a word, string
    word_c -- a word, string
    word_to_vec_map -- dictionary that maps words to their corresponding vectors. 
    
    Returns:
    best_word --  the word such that v_b - v_a is close to v_best_word - v_c, as measured by cosine similarity
    """
    
    # convert words to lower case
    word_a, word_b, word_c = word_a.lower(), word_b.lower(), word_c.lower()
    
    ### START CODE HERE ###
    # Get the word embeddings v_a, v_b and v_c (≈1-3 lines)
    e_a, e_b, e_c = None
    ### END CODE HERE ###
    
    words = word_to_vec_map.keys()
    max_cosine_sim = -100              # Initialize max_cosine_sim to a large negative number
    best_word = None                   # Initialize best_word with None, it will help keep track of the word to output

    # loop over the whole word vector set
    for w in words:        
        # to avoid best_word being one of the input words, pass on them.
        if w in [word_a, word_b, word_c] :
            continue
        
        ### START CODE HERE ###
        # Compute cosine similarity between the vector (e_b - e_a) and the vector ((w's vector representation) - e_c)  (≈1 line)
        cosine_sim = None
        
        # If the cosine_sim is more than the max_cosine_sim seen so far,
            # then: set the new max_cosine_sim to the current cosine_sim and the best_word to the current word (≈3 lines)
        if None:
            max_cosine_sim = None
            best_word = w
        ### END CODE HERE ###
        
    return best_word

Run the cell below to test your code, this may take 1-2 minutes.

In [ ]:
triads_to_try = [('italy', 'italian', 'spain'), ('india', 'delhi', 'japan'), ('man', 'woman', 'boy'), ('small', 'smaller', 'large')]

for triad in triads_to_try:
    print ('{} -> {} :: {} -> {}'.format( *triad, complete_analogy(*triad,word_to_vec_map)))

**Expected Output**:

<table>
    <tr>
        <td>
            **italy -> italian** ::
        </td>
        <td>
         spain -> spanish
        </td>
    </tr>
        <tr>
        <td>
            **india -> delhi** ::
        </td>
        <td>
         japan -> tokyo
        </td>
    </tr>
        <tr>
        <td>
            **man -> woman ** ::
        </td>
        <td>
         boy -> girl
        </td>
    </tr>
        <tr>
        <td>
            **small -> smaller ** ::
        </td>
        <td>
         large -> larger
        </td>
    </tr>
</table>

Once you get the correct expected output, please feel free to modify the input cells above to test your own analogies. Try to find some other analogy pairs that do work, but also find some where the algorithm doesn't give the right answer: For example, you can try small->smaller as big->?.  

# 3. Using Counter Factual Testing to Uncover Bias

One way to test for bias in a model is to use counter factual examples. In this section we study counter factual fairness in the model's predictions (for the word analogy tasks above). Specifically factual fairness asks the question: <i>How would the prediction change if the sensitive attribute referenced in the example were different?</i>

If the prediction changes with respect to a sensitive attribute, we consider this an indicator of a potential problem. In contrast to group-based notions of fairness (e.g., demographic parity, equality of odds), which seek to statistically equalize the model’s behavior for entire sensitive groups (we will see this in the final lab), counter factual fairness requires equal model behavior on individual counter factual pairs.

In this case we use a simple for of counter factual test where: we consider perturbations obtained by substituting tokens (words) associated with identity groups. e.g. man with woman, white with Asian, Asian with Black, white with black and so on. i.e. we design a word analogy set of triads where the counter factual tokens are the two of the three words.

In your experiment here you can investigate for any type of bias, for example those from the UK Law Protected Characteristics Equality Act 2010. You must however explore gender (need your discovered list of triads for gender  later). The Protected Characteristics are:

- Age
- Disability
- Gender reassignment
- Marriage and civil partnership
- Pregnancy and maternity
- Race
- Religion and belief
- Sex
- Sexual orientation


Although it is difficult using this type of limited language structure (word analogy) it is possible to uncover examples of bias.  For example try:

- 'white' is to 'boss' what 'black' is to ___
- 'women' is to 'nurse' what 'man' is to ___


**You need to:**

- explore word analogies for evidence of bias (minimum 10)
- capture a list of word analogies that provide evidence of various types of bias.


In [ ]:
### START CODE ###
triads_to_try = [('white', 'boss', 'black'), 
                 ('women', 'nurse', 'man')]
### END CODE ###

for triad in triads_to_try:
    print ('{} -> {} :: {} -> {}'.format( *triad, complete_analogy(*triad,word_to_vec_map)))

# 4. Removing Bias from Word Vector Models

In this section, you will examine gender biases (that you may well have found some evidence of using counter factuals above) that can be reflected in a word embedding model, and explore algorithms for reducing the bias.  In addition to learning about the topic of debiasing, this exercise will also help hone your intuition about what word vectors are doing. This section involves a bit of linear algebra, though you can complete it without being expert in linear algebra, by following and implementing the formulas given. 

**your task it to see how the GloVe word embeddings relate to gender:**
  
- You will need  compute a vector $g = e_{woman}-e_{man}$, where $e_{woman}$ represents the word vector corresponding to the word *woman*, and $e_{man}$ corresponds to the word vector corresponding to the word *man*. 

The resulting vector $g$ roughly encodes the concept of "gender". (You might get a more accurate representation if you compute $g_1 = e_{mother}-e_{father}$, $g_2 = e_{girl}-e_{boy}$, etc. and average over them. But just using $e_{woman}-e_{man}$ will give good enough results for now.) 


In [ ]:
#### START CODE #####
g = None
#### END CODE ######
print(g)

## 4.1 Word Similarity with Gender, $g$
Now, you will consider the cosine similarity of different words with $g$. Consider what a positive value of similarity means vs a negative cosine similarity. 

**Your task is to:**
- iterate over the list of names and print out the word and word similarity between each word (name) and the gender vector, $g$.
- try a list of your own names

In [ ]:
print ('List of names and their similarities with constructed vector:')

# girls and boys names
name_list = ['john', 'marie', 'sophie', 'ronaldo', 'priya', 'rahul', 'danielle', 'reza', 'katy', 'yasmin']

#### START CODE ####
for w in name_list:
    sim = 
    print None
    
new_list = None

for w in new_list:
    sim = 
    print None
#### END CODE #####

As you can see, female first names tend to have a positive cosine similarity with our constructed vector $g$, while male first names tend to have a negative cosine similarity. This is not suprising, and the result seems acceptable. 

**Your task is to try with some other words:**
- iterate over the provided word list
- second create a new word list of your own to investigate (approx. ten words)

In [ ]:
print('Other words and their similarities:')
word_list = ['lipstick', 'guns', 'science', 'arts', 'literature', 'warrior','doctor', 'tree', 'receptionist', 
             'technology',  'fashion', 'teacher', 'engineer', 'pilot', 'computer', 'singer']
#### START CODE ####
for w in None:
    sim = None
    print None
#### END CODE ###

#### START CODE ####

word_list_2 = None

for w in word_list_2:
    sim_2 = None
    print (None)
#### END CODE ###

Do you notice anything surprising? It is astonishing how these results reflect certain unhealthy gender stereotypes. For example, "computer" is closer to "man" while "literature" is closer to "woman". Ouch! 

We'll see below how to reduce the bias of these vectors, using an algorithm due to [2]. Note that some word pairs such as "actor"/"actress" or "grandmother"/"grandfather" should remain gender specific, while other words such as "receptionist" or "technology" should be neutralized, i.e. not be gender-related. You will have to treat these two type of words differently when debiasing.

[2] T. Bolukbasi, K.-W. Chang, J. Zou, V. Saligrama, and A. Kalai, “Man is to Computer Programmer as Woman is to Homemaker? Debiasing Word Embeddings,” Jul. 2016.[Link to Paper](https://papers.nips.cc/paper/6228-man-is-to-computer-programmer-as-woman-is-to-homemaker-debiasing-word-embeddings.pdf).

## 4.2 Neutralize Bias in Word Vectors 

The figure below should help you visualize what neutralizing does. If you're using a 50-dimensional word embedding, the 50 dimensional space can be split into two parts: The bias-direction $g$, and the remaining 49 dimensions, which we'll call $g_{\perp}$. In linear algebra, we say that the 49 dimensional $g_{\perp}$ is perpendicular (or "othogonal") to $g$, meaning it is at 90 degrees to $g$. The neutralization step takes a vector such as $e_{receptionist}$ and zeros out the component in the direction of $g$, giving us $e_{receptionist}^{debiased}$. 

Even though $g_{\perp}$ is 49 dimensional, given the limitations of what we can draw on a screen, we illustrate it using a 1 dimensional axis below. 

<img src="images/neutral.png" style="width:1000px;"/>
<caption><center> **Figure 3**: The word vector for "receptionist" represented before and after applying the neutralize operation. </center></caption>

**You will need to:**

Implement `neutralize()` to remove the bias of words such as "receptionist" or "scientist". Given an input embedding $e$, you can use the following formulas to compute $e^{debiased}$: 

$$e^{bias\_component} = \frac{e \cdot g}{||g||_2^2} * g\tag{2}$$
$$e^{debiased} = e - e^{bias\_component}\tag{3}$$


- Select word vector representation of `word`
- Compute Equation (2)
- Compute Equation (3)

**Hint:** You can make use of some or all of following Numpy functions (search for their usage):

```python
np.dot()
np.square()
np.linalg.norm()
np.sqrt()
```


In [ ]:
def neutralize(word, g, word_to_vec_map):
    """
    Removes the bias of "word" by projecting it on the space orthogonal to the bias axis. 
    This function ensures that gender neutral words are zero in the gender subspace.
    
    Arguments:
        word -- string indicating the word to debias
        g -- numpy-array of shape (50,), corresponding to the bias axis (such as gender)
        word_to_vec_map -- dictionary mapping words to their corresponding vectors.
    
    Returns:
        e_debiased -- neutralized word vector representation of the input "word"
    """
    
    ### START CODE HERE ###
    #Step1: Select word vector representation of "word". Use word_to_vec_map. (≈ 1 line)
    e = None
    
    # Compute e_biascomponent using the formula given above. (≈ 1 line)
    e_biascomponent = None
 
    # Neutralize e by substracting e_biascomponent from it 
    # e_debiased should be equal to its orthogonal projection. (≈ 1 line)
    e_debiased = None
    ### END CODE HERE ###
    
    return e_debiased

### 4.2.1 Testing your `neutralize` Function

Run the code below to test your neutralize function.

In [ ]:
e = "receptionist"
print("cosine similarity between " + e + " and g, before neutralizing: ", cosine_similarity(word_to_vec_map["receptionist"], g))

e_debiased = neutralize("receptionist", g, word_to_vec_map)
print("cosine similarity between " + e + " and g, after neutralizing: ", cosine_similarity(e_debiased, g))

**Expected Output**: The second result is essentially 0, up to numerical round-off (in the order of $10^{-17}$).


<table>
    <tr>
        <td>
            **cosine similarity between receptionist and g, before neutralizing:** :
        </td>
        <td>
         0.33077941750593737
        </td>
    </tr>
        <tr>
        <td>
            **cosine similarity between receptionist and g, after neutralizing:** :
        </td>
        <td>
         -3.415405087742776e-17
    </tr>
</table>

## 4.3 Equalization Algorithm to De-bias Word Vectors

Next, lets see how debiasing can also be applied to word pairs such as "actress" and "actor." Equalization is applied to pairs of words that you might want to have differ only through the gender property. As a concrete example, suppose that "actress" is closer to "babysit" than "actor." By applying neutralizing to "babysit" we can reduce the gender-stereotype associated with babysitting. But this still does not guarantee that "actor" and "actress" are equidistant from "babysit." The equalization algorithm takes care of this. 

The key idea behind equalization is to make sure that a particular pair of words are equidistant from the 49-dimensional $g_\perp$. The equalization step also ensures that the two equalized steps are now the same distance from $e_{receptionist}^{debiased}$, or from any other work that has been neutralized. In pictures, this is how equalization works: 

<img src="images/equalize10.png" style="width:800px;height:400px;"\>
<caption><center> **Figure 4**: The word vectors for "actor" and "actress" before and after applying the equalization operation. </center></caption>


The the linear algebra to do this is a bit more complex. (See [2] for more details if you wish). However summarised below are the key steps (equations) to complete the equalization algorithm: 

$$ \mu = \frac{e_{w1} + e_{w2}}{2}\tag{4}$$ 

$$ \mu_{B} = \frac {\mu \cdot \text{bias_axis}}{||\text{bias_axis}||_2^2} *\text{bias_axis}
\tag{5}$$ 

$$\mu_{\perp} = \mu - \mu_{B} \tag{6}$$

$$ e_{w1B} = \frac {e_{w1} \cdot \text{bias_axis}}{||\text{bias_axis}||_2^2} *\text{bias_axis}
\tag{7}$$ 
$$ e_{w2B} = \frac {e_{w2} \cdot \text{bias_axis}}{||\text{bias_axis}||_2^2} *\text{bias_axis}
\tag{8}$$


$$e_{w1B}^{corrected} = \sqrt{ |{1 - ||\mu_{\perp} ||^2_2} |} * \frac{e_{\text{w1B}} - \mu_B} {||(e_{w1} - \mu_{\perp}) - \mu_B)||_2} \tag{9}$$


$$e_{w2B}^{corrected} = \sqrt{ |{1 - ||\mu_{\perp} ||^2_2} |} * \frac{e_{\text{w2B}} - \mu_B} {||(e_{w2} - \mu_{\perp}) - \mu_B)||_2} \tag{10}$$

$$e_1 = e_{w1B}^{corrected} + \mu_{\perp} \tag{11}$$
$$e_2 = e_{w2B}^{corrected} + \mu_{\perp} \tag{12}$$


**You will need to:**

- Implement the function below. 
- Use the equations above each in turn to get the final equalized version of the pair of words.

Good luck!

In [ ]:
def equalize(pair, bias_axis, word_to_vec_map):
    """
    Debias gender specific words by following the equalize method described in the figure above.
    
    Arguments:
    pair -- pair of strings of gender specific words to debias, e.g. ("actress", "actor") 
    bias_axis -- numpy-array of shape (50,), vector corresponding to the bias axis, e.g. gender
    word_to_vec_map -- dictionary mapping words to their corresponding vectors
    
    Returns
    e_1 -- word vector corresponding to the first word
    e_2 -- word vector corresponding to the second word
    """
    
    ### START CODE HERE ###
    # Step 1: Select word vector representation of "word". Use word_to_vec_map. (≈ 2 lines)
    e_w1 = None
    e_w2 = None
    
    # Step 2: Compute the mean of e_w1 and e_w2 (≈ 1 line)
    mu = None

    # Step 3: Compute the projections of mu over the bias axis and the orthogonal axis (≈ 2 lines)
    mu_B = None

    # Step 4: Use equations (7) and (8) to compute e_w1B and e_w2B (≈2 lines)
    e_w1B = None
    e_w2B = None
    
    # Step 5: Adjust the Bias part of e_w1B and e_w2B using the formulas (9) and (10) given above (≈2 lines)
    corrected_e_w1B = None
    corrected_e_w2B = None
    
    # Step 6: Debias by equalizing e1 and e2 to the sum of their corrected projections (≈2 lines)
    e1 = None
    e2 = None
                                                                 
    ### END CODE HERE ###
    
    return e1, e2

### 4.3.1 Testing your `equalize` Function

Run the code below to test your equalise function.


In [ ]:
print("cosine similarities before equalizing:")
print("cosine_similarity(word_to_vec_map[\"man\"], gender) = ", cosine_similarity(word_to_vec_map["man"], g))
print("cosine_similarity(word_to_vec_map[\"woman\"], gender) = ", cosine_similarity(word_to_vec_map["woman"], g))
print()
e1, e2 = equalize(("man", "woman"), g, word_to_vec_map)
print("cosine similarities after equalizing:")
print("cosine_similarity(e1, gender) = ", cosine_similarity(e1, g))
print("cosine_similarity(e2, gender) = ", cosine_similarity(e2, g))

**Expected Output**:

cosine similarities before equalizing:
<table>
    <tr>
        <td>
            **cosine_similarity(word_to_vec_map["man"], gender)** =
        </td>
        <td>
         -0.117110957653
        </td>
    </tr>
        <tr>
        <td>
            **cosine_similarity(word_to_vec_map["woman"], gender)** =
        </td>
        <td>
         0.356666188463
        </td>
    </tr>
</table>

cosine similarities after equalizing:
<table>
    <tr>
        <td>
            **cosine_similarity(u1, gender)** =
        </td>
        <td>
         -0.18798753406068716
        </td>
    </tr>
        <tr>
        <td>
            **cosine_similarity(u2, gender)** =
        </td>
        <td>
         0.18798753406068716
        </td>
    </tr>
</table>

# 5. Improving Gender Bias and Fairness

These debiasing algorithms are very helpful for reducing bias, but are not perfect and do not eliminate all traces of bias. For example, one weakness of this implementation was that the bias direction $g$ was defined using only the pair of words _woman_ and _man_. As discussed earlier, if $g$ were defined by computing $g_1 = e_{woman} - e_{man}$; $g_2 = e_{mother} - e_{father}$; $g_3 = e_{girl} - e_{boy}$; and so on and averaging over them, you would obtain a better estimate of the "gender" direction in the 50 dimensional word embedding space. 

Using the word combinations that you discovered that pointed toward bais in the model re-run the word analogies on word vectors that have been de-biased using one of the two appropriate methods above.

**You need to:**

- implement the improved 'gender' direction by computing from an average of numerous word pairs $g_{av}$
- de-bias word embeddings using improved $g_{av}$ to be used in your word analogies 
 - Hint iterate over all words in the dictionary `word_to_vec_map` and apply the appropriate form of de-biasing
- re-run your counter factual triads from section 3
- re-run your word similarity with gender from section 4.1
               

In [ ]:
### START CODE ###

#Step 1: create imporved gender vector from averaging several gender pairs

None

#Step 2: debias word embeddings

None

#Step 3: re-run word anlogies (counter factuals).

None

# Step 4: re-run word-gender simlarities

None

### END CODE ###

<h1> 6. Critical Analysis, Reflection & Discussion</h1>

- What observations have you made when using for counter factuals for bias/fairness detection, specifically
    - its ability to detect potential bias
    - its ability to verify removal of bias
- What considerations might you make to the design of the improved Gender bias Vector to maximise fairness in the debiasing algorithm.
     - How do you think we should design the list of word pairs that generate $g_{av}$?
- What evidence have you found that this approach to de-biasing word vectors is effective?
- What are the limitations to this approach to de-biasing and why?
    - Can you think of approaches and actions that can help mitigate any limitations
- Think about and suggest an approach on how you might de-bias a non-linear model space.